In [3]:
# 📦 Imports
import pandas as pd
import glob

# 📁 Load raw Statcast data
files = sorted(glob.glob("../data/raw/statcast_*.csv"))
dfs = [pd.read_csv(file, low_memory=False) for file in files]
df = pd.concat(dfs, ignore_index=True)

# 📁 Load velocity grades (now includes 'season' and 'velocity_grade')
grades_raw = pd.read_csv("../data/processed/pitcher_velocity_grades.csv")

# ✅ Extract season from game_date (again for roles/team alignment)
df['season'] = pd.to_datetime(df['game_date']).dt.year
df = df.dropna(subset=['pitcher', 'inning'])

# ✅ Identify game_id per season
df['game_id'] = df['game_date'].astype(str) + "_" + df['home_team'] + "_" + df['away_team']
df = df.sort_values(by=['season', 'game_id', 'inning', 'pitch_number'])

# ✅ Detect starter appearances (first pitcher per team per game)
first_pitchers = df.groupby(['season', 'game_id']).first().reset_index()[['season', 'game_id', 'pitcher']]
starter_counts = first_pitchers.groupby(['season', 'pitcher']).size().reset_index(name='starts')
total_counts = df.groupby(['season', 'pitcher']).size().reset_index(name='total')

# 🧠 Assign role
roles = pd.merge(total_counts, starter_counts, on=['season', 'pitcher'], how='left').fillna(0)
roles['starts'] = roles['starts'].astype(int)
roles['role'] = roles.apply(lambda row: 'Starter' if row['starts'] >= 5 else 'Reliever', axis=1)

# 🧾 Get team per pitcher per season
pitcher_teams = df.groupby(['season', 'pitcher'])['home_team'].first().reset_index()
pitcher_teams.columns = ['season', 'pitcher', 'team']

# 🧮 Expand velocity grades by letter using one-hot encoding
grade_dummies = pd.get_dummies(grades_raw['velocity_grade'])
grades_expanded = pd.concat([grades_raw[['season', 'pitcher']], grade_dummies], axis=1)

# 🧮 Sum grades per pitcher per season
grades_by_pitcher = grades_expanded.groupby(['season', 'pitcher']).sum(numeric_only=True).reset_index()

# 🔗 Merge role + team into grade data
grades = grades_by_pitcher.merge(pitcher_teams, on=['season', 'pitcher'], how='left')
grades = grades.merge(roles[['season', 'pitcher', 'role']], on=['season', 'pitcher'], how='left')
grades = grades.dropna(subset=['season'])  # just in case

# 🧮 Scoring weights
grade_weights = {'A': 4, 'B': 3, 'C': 2, 'D': 1, 'F': 0}
for grade, weight in grade_weights.items():
    if grade in grades.columns:
        grades[grade] = grades[grade] * weight
    else:
        grades[grade] = 0

grades['role_score'] = grades[['A', 'B', 'C', 'D', 'F']].sum(axis=1)

# 📊 Aggregate scores per team/season/role
team_scores = grades.groupby(['season', 'team', 'role'])['role_score'].sum().reset_index()

# 📐 Pivot into wide format: one row per team-season
pivoted = team_scores.pivot_table(index=['team', 'season'], columns='role', values='role_score', fill_value=0).reset_index()
pivoted['combined_score'] = pivoted.get('Starter', 0) + pivoted.get('Reliever', 0)

# 💾 Save result
pivoted.to_csv("../data/processed/team_seasonal_pitching_scores.csv", index=False)
print("✅ Saved to ../data/processed/team_seasonal_pitching_scores.csv")

# 🔍 Preview
pivoted.sort_values(['season', 'combined_score'], ascending=[False, False]).head()


✅ Saved to ../data/processed/team_seasonal_pitching_scores.csv


role,team,season,Reliever,Starter,combined_score
72,NYM,2025,285.0,53.0,338.0
20,CHC,2025,269.0,63.0,332.0
36,CWS,2025,264.0,38.0,302.0
115,TOR,2025,265.0,36.0,301.0
111,TEX,2025,277.0,23.0,300.0
